## Prerequisites - Environment Setup

Before starting this workshop, you need to set up a Python virtual environment. Follow these steps:

### Step 1: Create Virtual Environment
From the **root folder** of this repository, run:
```bash
python -m venv .venv
```

### Step 2: Activate Virtual Environment
- **Windows**: `.venv\Scripts\activate`
- **macOS/Linux**: `source .venv/bin/activate`

### Step 3: Upgrade pip
```bash
pip install --upgrade pip
```

### Step 4: Install Jupyter Kernel Support
```bash
pip install ipykernel
```

### Step 5: Select Virtual Environment in VS Code
1. Click on the **kernel selector** in the top-right corner of this notebook
2. Select **"Select Another Kernel..."**
3. Choose **"Python Environments..."**
4. Select the `.venv` environment you just created

### Step 6: Verify Setup
Once you've selected the correct kernel, you can proceed with the workshop. The first code cell will install the required dependencies.

---

# Semantic Kernel Agents Workshop: From Simple to Advanced

Welcome to the Semantic Kernel Agents Workshop! This hands-on tutorial will take you on an exciting journey from basic agent concepts to advanced multi-agent systems using Microsoft's Semantic Kernel framework.

## What You'll Build

1. **Generic Agent** - Start simple with a basic conversational agent
2. **Azure AI Foundry Agent** - Level up with cloud-powered AI capabilities using Semantic Kernel
3. **Group Chat System** - Master advanced multi-agent orchestration and collaboration

## Why Semantic Kernel?

Semantic Kernel is Microsoft's framework that enables:
- **Plugin Architecture**: Modular, extensible agent capabilities
- **AI Orchestration**: Seamless integration with multiple AI services
- **Function Calling**: Native integration with tools and APIs
- **Memory Management**: Sophisticated conversation and context handling
- **Enterprise Ready**: Built for production environments

## Quick Setup

This section sets up everything you need to run Semantic Kernel agents with minimal friction.

### Prerequisites

Before we start, ensure you have:
- Python 3.8+ installed
- Virtual environment activated (recommended)
- Azure OpenAI endpoint and API key
- Basic familiarity with async/await patterns

### Learning Path

We'll progress from simple to advanced:
1. **Foundation**: Basic Semantic Kernel setup and simple agents
2. **Evolution**: Adaptive agents with memory and learning
3. **Orchestration**: Multi-agent systems with specialized roles
4. **Mastery**: Production-ready collaborative AI systems

Let's begin this exciting journey!

## Step 1: Install Dependencies & Setup

Let's set up your Semantic Kernel environment with all required dependencies.

In [ ]:
# Step 1: Install Dependencies & Environment Setup
# Comprehensive setup for Semantic Kernel agents

import os
import sys
import subprocess
import pathlib
from pathlib import Path

nb_dir = pathlib.Path().resolve()
project_root = nb_dir.parents[2] if (len(nb_dir.parents) >= 2) else nb_dir
sk_dir = nb_dir  # this notebook lives in Backend/python/sk
shared_dir = sk_dir.parent / "shared"
req_file = sk_dir / "requirements.txt"

print("Environment Detection:")
print(f"   Notebook directory: {nb_dir}")
print(f"   Project root: {project_root}")
print(f"   SK directory: {sk_dir}")
print(f"   Shared library: {shared_dir}")
print(f"   Requirements file: {req_file}")

def run_command(cmd):
    """Execute a command and handle errors gracefully."""
    print(f"\nExecuting: {cmd}")
    result = subprocess.run(cmd, shell=True, text=True, capture_output=True)
    if result.returncode != 0:
        print(f"Error: {result.stderr}")
        return False
    if result.stdout:
        print(f"Success: {result.stdout[:200]}...")
    return True

def install_dependencies():
    """Install all required dependencies for Semantic Kernel."""
    print("\nInstalling Semantic Kernel Dependencies...")
    
    try:
        # Try using IPython magic commands first (preferred in notebooks)
        import IPython
        get_ipython()  # Verify we're in a Jupyter environment
        
        print("Using IPython magic commands...")
        
        # Install requirements
        if req_file.exists():
            get_ipython().run_line_magic("pip", f"install -r {req_file}")
            print("✓ Requirements installed successfully!")
        else:
            print("! requirements.txt not found, installing core dependencies...")
            # Core Semantic Kernel dependencies
            core_deps = [
                "semantic-kernel",
                "openai",
                "azure-identity", 
                "python-dotenv",
                "asyncio"
            ]
            for dep in core_deps:
                get_ipython().run_line_magic("pip", f"install {dep}")
            print("✓ Core dependencies installed!")
        
        # Install shared library if available
        if (shared_dir / "setup.py").exists():
            get_ipython().run_line_magic("pip", f"install -e {shared_dir}")
            print("✓ Shared library installed!")
        else:
            print("! Shared library not found, skipping...")
            
    except Exception as e:
        print(f"! IPython magic not available: {e}")
        print("Falling back to subprocess commands...")
        
        # Fallback to subprocess
        if req_file.exists():
            success = run_command(f'python -m pip install -r "{req_file}"')
            if not success:
                return False
        
        if (shared_dir / "setup.py").exists():
            success = run_command(f'python -m pip install -e "{shared_dir}"')
            if not success:
                return False
    
    return True

def verify_installation():
    """Verify that Semantic Kernel is properly installed."""
    print("\nVerifying Semantic Kernel installation...")
    
    try:
        import semantic_kernel as sk
        print(f"✓ Semantic Kernel version: {sk.__version__}")
        
        # Test Azure OpenAI imports
        from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
        print("✓ Azure OpenAI connector available")
        
        # Test core components
        from semantic_kernel.core_plugins import ConversationSummaryPlugin
        print("✓ Core plugins available")
        
        return True
        
    except ImportError as e:
        print(f"! Import error: {e}")
        return False

# Execute the installation process
print("Starting Semantic Kernel setup...")
if install_dependencies():
    if verify_installation():
        print("\n✓ Semantic Kernel setup completed successfully!")
        print("Ready to build amazing AI agents!")
    else:
        print("\n! Installation verification failed")
else:
    print("\n! Dependency installation failed")

In [ ]:
# Step 2: Configure Azure OpenAI Keys
# Simple and secure setup for Azure OpenAI services (recommended for beginners)

import os
from dotenv import load_dotenv

# Load existing environment variables
load_dotenv()

print("Checking Azure OpenAI configuration...")
print("Make sure you have a .env file with the required keys!")

# Required Azure OpenAI environment variables
required_vars = {
    "AZURE_OPENAI_ENDPOINT": "Your Azure OpenAI endpoint URL",
    "AZURE_OPENAI_API_KEY": "Your Azure OpenAI API key", 
    "AZURE_OPENAI_DEPLOYMENT_NAME": "Your model deployment name (e.g., gpt-4o-mini)"
}

# Check if all required variables are present
missing_vars = []
for var_name, description in required_vars.items():
    value = os.getenv(var_name)
    if value:
        print(f"✓ {var_name}: Found")
    else:
        print(f"✗ {var_name}: Missing")
        missing_vars.append(var_name)

if missing_vars:
    print(f"\n! Missing required environment variables: {', '.join(missing_vars)}")
    print("\nTo fix this, create a .env file in this directory with:")
    print("AZURE_OPENAI_ENDPOINT=https://your-resource.openai.azure.com/")
    print("AZURE_OPENAI_API_KEY=your_api_key_here")
    print("AZURE_OPENAI_DEPLOYMENT_NAME=gpt-4o-mini")
    print("\nThen restart this notebook kernel and run this cell again.")
    raise ValueError("Required Azure OpenAI configuration missing!")

# Show successful configuration
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

print(f"\n✓ Azure OpenAI configuration loaded successfully!")
print(f"Endpoint: {endpoint}")
print(f"Deployment: {deployment}")
print("Ready to create your Semantic Kernel Agent!")

In [ ]:
# Step 3: Import Required Libraries
# Import all the Semantic Kernel libraries we'll use for our agents

import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.core_plugins.text_memory_plugin import TextMemoryPlugin
from semantic_kernel.memory.semantic_text_memory import SemanticTextMemory
from semantic_kernel.connectors.ai.open_ai import AzureTextEmbedding
from semantic_kernel.memory.volatile_memory_store import VolatileMemoryStore

# Additional imports for our agent system
import asyncio
import json
from typing import List, Dict, Any
from datetime import datetime

print("All Semantic Kernel libraries imported successfully!")
print("Ready to build our agent system!")

In [ ]:
# Step 4: Create Your First Semantic Kernel Agent
# This cell creates a simple agent that can have conversations

class GenericAgent:
    def __init__(self, name: str, description: str, deployment_name: str = None):
        """Initialize a generic agent with Semantic Kernel"""
        self.name = name
        self.description = description
        
        # Create a new kernel for this agent
        self.kernel = sk.Kernel()
        
        # Use deployment name from environment if not provided
        if deployment_name is None:
            deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
        
        # Add Azure OpenAI chat completion service
        chat_service = AzureChatCompletion(
            deployment_name=deployment_name,
            endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            api_key=os.getenv("AZURE_OPENAI_API_KEY")
        )
        
        self.kernel.add_service(chat_service)
        
    async def chat(self, message: str) -> str:
        """Send a message to the agent and get a response"""
        # Create a system message with the agent's description
        system_message = f"You are {self.name}. {self.description}"
        
        # Combine system message with user message
        full_prompt = f"{system_message}\n\nUser: {message}\nAssistant:"
        
        # Get response from the kernel
        response = await self.kernel.invoke_prompt(full_prompt)
        return str(response)

# Create our first agent
generic_agent = GenericAgent(
    name="Assistant", 
    description="A helpful AI assistant that can answer questions and have conversations."
)

print("Generic Agent created successfully!")
print(f"Agent name: {generic_agent.name}")
print(f"Description: {generic_agent.description}")
print("Ready to chat!")

### Test Your Agent
You've created your first Semantic Kernel agent! Let's test it to make sure everything is working.

**Note:** The cell below will test the agent by sending it a simple message. If you see an error:
- Check that your Azure OpenAI configuration is correct
- Make sure your API key has access to the deployment
- Verify your deployment name is spelled correctly

**Try asking questions like:**
- "What is artificial intelligence?"
- "Explain machine learning in simple terms"
- "What can you help me with?"

This basic agent can handle general conversations. In the next sections, we'll create more specialized agents with specific roles and capabilities!

#### Test the Generic Agent

Run this cell to test your agent:

Change the message below to test different questions:

In [ ]:
# Test the Generic Agent
# Run this cell to test your agent with a simple question

async def test_agent():
    """Test function to chat with our agent"""
    try:
        # Send a test message to the agent
        test_message = "Hello! What is artificial intelligence?"
        print(f"User: {test_message}")
        
        # Get response from agent
        response = await generic_agent.chat(test_message)
        print(f"Agent ({generic_agent.name}): {response}")
        
        print("\nAgent test completed successfully!")
        print("You can now modify the test_message above to ask different questions.")
        
    except Exception as e:
        print(f"Error testing agent: {e}")
        print("\nTroubleshooting tips:")
        print("1. Check your Azure OpenAI configuration")
        print("2. Verify your API key and endpoint")
        print("3. Make sure your deployment name is correct")

# Run the test
await test_agent()

In [ ]:
# Create Specialized Agents
# Now let's create agents with specific roles and expertise

class SpecializedAgent(GenericAgent):
    def __init__(self, name: str, role: str, expertise: str, personality: str = "professional and helpful"):
        # Create a detailed description based on role and expertise  
        description = f"""You are {name}, a {role} with expertise in {expertise}. 
        Your personality is {personality}.
        
        Guidelines for responses:
        - Focus on your area of expertise: {expertise}
        - Maintain your role as a {role}
        - Be {personality} in all interactions
        - Provide practical, actionable advice when possible
        - If asked about topics outside your expertise, acknowledge the limitation but offer what help you can
        """
        
        # Initialize the parent GenericAgent with our specialized description
        super().__init__(name, description)
        
        # Store additional attributes
        self.role = role
        self.expertise = expertise
        self.personality = personality

# Create specialized agents
print("Creating specialized agents...")

# Technical expert
tech_expert = SpecializedAgent(
    name="TechExpert",
    role="Senior Software Engineer", 
    expertise="Python programming, software architecture, and best practices",
    personality="analytical and detail-oriented"
)

# Business consultant  
business_expert = SpecializedAgent(
    name="BusinessGuru",
    role="Business Strategy Consultant",
    expertise="market analysis, business strategy, and digital transformation", 
    personality="strategic and results-focused"
)

# Creative expert
creative_expert = SpecializedAgent(
    name="CreativeMind", 
    role="Creative Director",
    expertise="creative writing, marketing campaigns, and content strategy",
    personality="imaginative and inspiring"
)

print("Specialized agents created successfully!")
print(f"1. {tech_expert.name} - {tech_expert.role}")
print(f"2. {business_expert.name} - {business_expert.role}") 
print(f"3. {creative_expert.name} - {creative_expert.role}")
print("\nEach agent has specialized knowledge and a unique personality!")

In [ ]:
# Test Specialized Agents
# Let's test each specialized agent with questions in their area of expertise

async def test_specialized_agents():
    """Test all specialized agents with relevant questions"""
    
    # Test questions for each agent
    test_cases = [
        {
            "agent": tech_expert,
            "question": "What are the best practices for writing clean Python code?"
        },
        {
            "agent": business_expert, 
            "question": "How should a startup approach digital transformation?"
        },
        {
            "agent": creative_expert,
            "question": "Create a catchy tagline for a new eco-friendly water bottle"
        }
    ]
    
    print("Testing specialized agents...\n")
    
    for i, test_case in enumerate(test_cases, 1):
        agent = test_case["agent"]
        question = test_case["question"]
        
        try:
            print(f"Test {i}: {agent.name} ({agent.role})")
            print(f"Question: {question}")
            print("-" * 50)
            
            # Get response from the agent
            response = await agent.chat(question)
            print(f"Response: {response}")
            print("=" * 70)
            print()
            
        except Exception as e:
            print(f"Error testing {agent.name}: {e}")
            print("=" * 70)
            print()
    
    print("Specialized agent testing completed!")
    print("\nNotice how each agent responds according to their role and expertise!")

# Run the tests
await test_specialized_agents()

In [ ]:
# Try Your Own Questions
# Modify the questions below to test the agents with your own prompts

async def test_custom_questions():
    """Test agents with custom questions - modify the questions below!"""
    
    # Modify these questions to test different scenarios
    custom_tests = [
        {
            "agent": tech_expert,
            "question": "How do I optimize Python code for better performance?"  # Change this question
        },
        {
            "agent": business_expert,
            "question": "What metrics should I track for a SaaS business?"  # Change this question
        },
        {
            "agent": creative_expert, 
            "question": "Write a creative social media post about AI"  # Change this question
        }
    ]
    
    print("Testing custom questions...\n")
    
    for i, test in enumerate(custom_tests, 1):
        agent = test["agent"]
        question = test["question"]
        
        try:
            print(f"Custom Test {i}: {agent.name}")
            print(f"Your Question: {question}")
            print("-" * 50)
            
            response = await agent.chat(question)
            print(f"Response: {response}")
            print("=" * 70)
            print()
            
        except Exception as e:
            print(f"Error with {agent.name}: {e}")
            print("=" * 70)
            print()

# Run your custom tests
await test_custom_questions()

## Multi-Agent Group Chat with Semantic Kernel

Now for the exciting part! Let's create a **Group Chat Orchestrator** that can coordinate multiple agents working together on complex tasks.

### How Group Chat Works:
1. **User poses a question** - You ask a complex question that might benefit from multiple perspectives
2. **Orchestrator analyzes** - The orchestrator determines which agents are most relevant
3. **Agents collaborate** - Multiple agents contribute their expertise to the discussion
4. **Synthesized response** - The orchestrator combines insights into a comprehensive answer

### Example Scenarios:
- **Product Launch Strategy**: Business expert handles market analysis, creative expert designs marketing, tech expert addresses technical requirements
- **Website Development**: Tech expert handles architecture, creative expert designs UI/UX, business expert considers user needs
- **Problem Solving**: Multiple perspectives lead to more comprehensive solutions

This approach mimics real-world collaboration where different experts contribute their specialized knowledge!

### Create the Group Chat Orchestrator

In [ ]:
# Group Chat Orchestrator
# This orchestrator coordinates multiple agents to work together on complex tasks

class GroupChatOrchestrator:
    def __init__(self, agents: List[GenericAgent]):
        """Initialize the orchestrator with a list of available agents"""
        self.agents = {agent.name: agent for agent in agents}
        self.conversation_history = []
        
        # Create the orchestrator's own kernel for coordination decisions
        self.kernel = sk.Kernel()
        
        # Add Azure OpenAI service for the orchestrator
        chat_service = AzureChatCompletion(
            deployment_name=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
            endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            api_key=os.getenv("AZURE_OPENAI_API_KEY")
        )
        self.kernel.add_service(chat_service)
    
    async def _select_relevant_agents(self, question: str, max_agents: int = 3) -> List[str]:
        """Use AI to determine which agents are most relevant for the question"""
        
        # Create agent descriptions for selection
        agent_descriptions = []
        for name, agent in self.agents.items():
            if hasattr(agent, 'role') and hasattr(agent, 'expertise'):
                desc = f"- {name}: {agent.role} with expertise in {agent.expertise}"
            else:
                desc = f"- {name}: {agent.description}"
            agent_descriptions.append(desc)
        
        agent_list = "\n".join(agent_descriptions)
        
        selection_prompt = f"""
        Given this question: "{question}"
        
        Available agents:
        {agent_list}
        
        Select the {max_agents} most relevant agents to answer this question. 
        Consider which agents' expertise would be most valuable.
        
        Respond with ONLY the agent names, separated by commas (e.g., "TechExpert, BusinessGuru").
        """
        
        try:
            response = await self.kernel.invoke_prompt(selection_prompt.strip())
            selected_names = [name.strip() for name in str(response).split(',')]
            
            # Validate agent names and limit to max_agents
            valid_names = [name for name in selected_names if name in self.agents][:max_agents]
            
            # If no valid names, default to first available agents
            if not valid_names:
                valid_names = list(self.agents.keys())[:max_agents]
                
            return valid_names
            
        except Exception as e:
            print(f"Error selecting agents: {e}")
            # Fallback to first available agents
            return list(self.agents.keys())[:max_agents]
    
    async def group_chat(self, question: str, max_agents: int = 3) -> Dict[str, Any]:
        """Orchestrate a group chat session to answer the question"""
        
        print(f"Question: {question}")
        print("=" * 60)
        
        # Select relevant agents
        print("Selecting relevant agents...")
        selected_agent_names = await self._select_relevant_agents(question, max_agents)
        
        print(f"Selected agents: {', '.join(selected_agent_names)}")
        print("-" * 60)
        
        # Collect responses from selected agents
        agent_responses = {}
        
        for agent_name in selected_agent_names:
            if agent_name in self.agents:
                try:
                    print(f"\nGetting response from {agent_name}...")
                    agent = self.agents[agent_name]
                    response = await agent.chat(question)
                    agent_responses[agent_name] = response
                    
                    # Display the response
                    if hasattr(agent, 'role'):
                        print(f"{agent_name} ({agent.role}):")
                    else:
                        print(f"{agent_name}:")
                    print(f"{response}")
                    print("-" * 40)
                    
                except Exception as e:
                    print(f"Error getting response from {agent_name}: {e}")
                    agent_responses[agent_name] = f"Error: {str(e)}"
        
        # Create synthesis prompt
        response_summary = []
        for name, response in agent_responses.items():
            agent = self.agents[name]
            role = getattr(agent, 'role', 'Agent')
            response_summary.append(f"**{name} ({role})**: {response}")
        
        synthesis_prompt = f"""
        Question: {question}
        
        Expert responses:
        {chr(10).join(response_summary)}
        
        Please synthesize these expert opinions into a comprehensive, well-structured answer.
        Combine the different perspectives and highlight key insights from each expert.
        Make the final answer cohesive and actionable.
        """
        
        try:
            print("\nSynthesizing group response...")
            synthesis = await self.kernel.invoke_prompt(synthesis_prompt.strip())
            
            print("\n" + "=" * 60)
            print("SYNTHESIZED GROUP RESPONSE:")
            print("=" * 60)
            print(str(synthesis))
            
        except Exception as e:
            print(f"Error creating synthesis: {e}")
            synthesis = "Error creating synthesis. Please review individual agent responses above."
        
        # Store conversation history
        conversation_record = {
            "question": question,
            "selected_agents": selected_agent_names,
            "agent_responses": agent_responses,
            "synthesis": str(synthesis),
            "timestamp": datetime.now().isoformat()
        }
        
        self.conversation_history.append(conversation_record)
        
        return conversation_record

# Create the group chat orchestrator with our specialized agents
orchestrator = GroupChatOrchestrator([
    generic_agent,
    tech_expert, 
    business_expert,
    creative_expert
])

print("Group Chat Orchestrator created successfully!")
print(f"Available agents: {list(orchestrator.agents.keys())}")
print("Ready for group collaboration!")

### Test Group Chat - Product Launch Example

In [ ]:
# Test Group Chat - Product Launch Strategy
# Watch how multiple agents collaborate to create a comprehensive product launch strategy

async def test_group_chat():
    """Test the group chat with a complex business scenario"""
    
    # Complex question that benefits from multiple perspectives
    complex_question = """
    We're launching a new AI-powered mobile app that helps people learn languages. 
    We need a complete strategy covering technical architecture, business model, 
    and marketing approach. What should we consider for a successful launch?
    """
    
    print("Testing Group Chat with Product Launch Scenario")
    print("This will demonstrate how multiple agents collaborate...")
    print()
    
    try:
        # Run the group chat
        result = await orchestrator.group_chat(complex_question, max_agents=3)
        
        print("\n" + "=" * 60)
        print("GROUP CHAT COMPLETED SUCCESSFULLY!")
        print("=" * 60)
        print(f"Question answered by: {', '.join(result['selected_agents'])}")
        print(f"Conversation saved at: {result['timestamp']}")
        
    except Exception as e:
        print(f"Error in group chat: {e}")
        print("\nTroubleshooting tips:")
        print("1. Check your Azure OpenAI configuration")
        print("2. Verify all agents are properly initialized")

# Run the group chat test
await test_group_chat()

In [ ]:
# Try Your Own Group Chat Question
# Modify the question below to test the group chat with your own scenario

async def test_custom_group_chat():
    """Test group chat with your own question"""
    
    # MODIFY THIS QUESTION to test different scenarios:
    your_question = """
    How can we improve team productivity in a remote software development team? 
    Consider technical tools, management practices, and team culture.
    """
    
    print("Testing Group Chat with Your Custom Question")
    print("Feel free to modify the question above to explore different topics!")
    print()
    
    try:
        # Run group chat with your question
        result = await orchestrator.group_chat(your_question, max_agents=3)
        
        print("\n" + "=" * 60) 
        print("YOUR CUSTOM GROUP CHAT COMPLETED!")
        print("=" * 60)
        print(f"Agents consulted: {', '.join(result['selected_agents'])}")
        
    except Exception as e:
        print(f"Error in custom group chat: {e}")

# Run your custom group chat
await test_custom_group_chat()

### Advanced Features and Memory

In [ ]:
# Add Memory to Agents
# Let's enhance our agents with memory capabilities using Semantic Kernel's memory features

class MemoryEnabledAgent(GenericAgent):
    def __init__(self, name: str, description: str, deployment_name: str = None):
        """Initialize an agent with memory capabilities"""
        super().__init__(name, description, deployment_name)
        
        # Add memory capabilities
        self.memory_store = VolatileMemoryStore()
        
        # Add text embedding service for memory
        embedding_service = AzureTextEmbedding(
            deployment_name="text-embedding-ada-002",  # Use a standard embedding model
            endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            api_key=os.getenv("AZURE_OPENAI_API_KEY")
        )
        
        self.kernel.add_service(embedding_service)
        
        # Create semantic memory
        self.memory = SemanticTextMemory(self.memory_store, embedding_service)
        
        # Add memory plugin to kernel
        memory_plugin = TextMemoryPlugin(self.memory)
        self.kernel.add_plugin(memory_plugin, "memory")
        
        # Store for conversation history
        self.conversation_history = []
        self.memory_collection = f"{name}_memory"
    
    async def remember(self, key: str, text: str, description: str = ""):
        """Store information in the agent's memory"""
        try:
            await self.memory.save_information(
                collection=self.memory_collection,
                text=text,
                id=key,
                description=description
            )
            print(f"{self.name} remembered: {key}")
        except Exception as e:
            print(f"Error saving memory: {e}")
    
    async def recall(self, query: str, limit: int = 3) -> List[str]:
        """Recall relevant information from memory"""
        try:
            results = await self.memory.search(
                collection=self.memory_collection,
                query=query,
                limit=limit
            )
            return [result.text for result in results]
        except Exception as e:
            print(f"Error recalling memory: {e}")
            return []
    
    async def chat_with_memory(self, message: str) -> str:
        """Chat with the agent using memory context"""
        # Recall relevant memories
        relevant_memories = await self.recall(message)
        
        # Build context with memories
        memory_context = ""
        if relevant_memories:
            memory_context = f"\nRelevant context from previous conversations:\n"
            memory_context += "\n".join([f"- {memory}" for memory in relevant_memories])
        
        # Create prompt with memory context
        system_message = f"You are {self.name}. {self.description}"
        full_prompt = f"{system_message}{memory_context}\n\nUser: {message}\nAssistant:"
        
        # Get response
        response = await self.kernel.invoke_prompt(full_prompt)
        
        # Store this conversation in memory
        conversation_key = f"conversation_{len(self.conversation_history)}"
        conversation_text = f"User asked: {message}. I responded: {str(response)}"
        await self.remember(conversation_key, conversation_text, f"Conversation on {datetime.now()}")
        
        # Store in conversation history
        self.conversation_history.append({
            "user": message,
            "assistant": str(response),
            "timestamp": datetime.now().isoformat()
        })
        
        return str(response)

# Create a memory-enabled expert agent
print("Creating memory-enabled agent...")

try:
    memory_expert = MemoryEnabledAgent(
        name="MemoryExpert",
        description="An AI assistant with persistent memory that can remember previous conversations and build on past knowledge."
    )
    print("Memory-enabled agent created successfully!")
    print(f"Agent: {memory_expert.name}")
    print("This agent can remember previous conversations and use that context in future responses.")
    
except Exception as e:
    print(f"Error creating memory-enabled agent: {e}")
    print("Note: Memory features require text embedding capabilities.")
    print("You may need to check your Azure OpenAI embedding model deployment.")

In [ ]:
# Test Memory Capabilities
# Let's test how the agent remembers and uses previous conversation context

async def test_memory_agent():
    """Test the memory capabilities of our enhanced agent"""
    
    print("Testing Memory-Enabled Agent")
    print("=" * 40)
    
    try:
        # First conversation - establish some context
        print("Conversation 1: Teaching the agent about a project")
        message1 = "I'm working on a Python project called 'DataFlow' that processes customer data using pandas and machine learning."
        response1 = await memory_expert.chat_with_memory(message1)
        print(f"User: {message1}")
        print(f"Agent: {response1}")
        print("-" * 40)
        
        # Second conversation - see if it remembers
        print("\nConversation 2: Asking follow-up questions")
        message2 = "What libraries would you recommend for my project?"
        response2 = await memory_expert.chat_with_memory(message2)
        print(f"User: {message2}")
        print(f"Agent: {response2}")
        print("-" * 40)
        
        # Third conversation - test memory recall
        print("\nConversation 3: Testing specific memory")
        message3 = "What was the name of my project again?"
        response3 = await memory_expert.chat_with_memory(message3)
        print(f"User: {message3}")
        print(f"Agent: {response3}")
        print("-" * 40)
        
        print(f"\nMemory test completed! The agent had {len(memory_expert.conversation_history)} conversations.")
        print("Notice how the agent remembers previous context and builds on it!")
        
    except Exception as e:
        print(f"Error testing memory agent: {e}")
        print("This might be due to embedding model requirements.")

# Run the memory test
await test_memory_agent()

## Advanced Workshop Challenge

Now that you've learned the fundamentals, here's a challenge to extend your skills!

### Challenge: Create Your Own Agent System

**Your Mission**: Build a specialized agent system for a domain of your choice. Here are some ideas:

1. **Educational Tutoring System**: Create subject-specific tutors (Math, Science, History) that can work together to help students
2. **Travel Planning Agency**: Build agents for different aspects (Budget Planner, Activity Recommender, Cultural Guide)
3. **Health & Wellness Team**: Create agents for nutrition, fitness, mental health, and medical information
4. **Creative Writing Studio**: Build agents for different writing aspects (Plot, Character Development, Dialogue, Editing)

### Your Challenge Requirements:
- Create at least 3 specialized agents with distinct roles
- Implement group chat coordination between them
- Add memory capabilities to at least one agent
- Test your system with realistic scenarios

In [ ]:
# Challenge Solution Template
# Use this template to build your own specialized agent system

# Step 1: Define your domain and agent roles
DOMAIN = "Educational Tutoring"  # Change this to your chosen domain

# Define your specialized agents
class CustomSpecializedAgent(MemoryEnabledAgent):
    def __init__(self, name: str, subject: str, teaching_style: str, grade_level: str = "high school"):
        description = f"""You are {name}, an expert {subject} tutor for {grade_level} students.
        Your teaching style is {teaching_style}.
        
        Teaching Guidelines:
        - Explain concepts clearly and appropriately for {grade_level} level
        - Use a {teaching_style} approach
        - Provide examples and practice problems when helpful
        - Encourage questions and critical thinking
        - Build on previous knowledge when possible
        """
        
        super().__init__(name, description)
        self.subject = subject
        self.teaching_style = teaching_style
        self.grade_level = grade_level

# Step 2: Create your specialized agents
print(f"Creating {DOMAIN} Agent System...")

# Example: Educational tutors (modify for your domain)
math_tutor = CustomSpecializedAgent(
    name="MathMentor",
    subject="Mathematics", 
    teaching_style="step-by-step and visual",
    grade_level="high school"
)

science_tutor = CustomSpecializedAgent(
    name="ScienceGuide", 
    subject="Science",
    teaching_style="hands-on and experimental",
    grade_level="high school"
)

english_tutor = CustomSpecializedAgent(
    name="LiteraturePro",
    subject="English Literature", 
    teaching_style="discussion-based and analytical",
    grade_level="high school"
)

# Step 3: Create your custom group chat orchestrator
custom_orchestrator = GroupChatOrchestrator([
    math_tutor,
    science_tutor, 
    english_tutor
])

print("Custom agent system created!")
print(f"Domain: {DOMAIN}")
print(f"Agents: {list(custom_orchestrator.agents.keys())}")

# Step 4: Test your system
async def test_custom_system():
    """Test your custom agent system"""
    
    # Modify this test question for your domain
    test_question = """
    I need help understanding how mathematics and science connect in real-world applications.
    Can you explain how calculus is used in physics, with some concrete examples?
    """
    
    print(f"\nTesting {DOMAIN} System")
    print("=" * 50)
    
    try:
        result = await custom_orchestrator.group_chat(test_question, max_agents=3)
        
        print("\n" + "=" * 50)
        print("CUSTOM SYSTEM TEST COMPLETED!")
        print("=" * 50)
        
    except Exception as e:
        print(f"Error testing custom system: {e}")

# Run your custom system test
await test_custom_system()

# Step 5: Add your own innovations
print(f"\n{DOMAIN} Agent System is ready!")
print("\nNext steps to enhance your system:")
print("1. Add more specialized agents")
print("2. Implement domain-specific memory features")
print("3. Create custom conversation flows")
print("4. Add external data sources or APIs")
print("5. Implement user preference learning")

In [ ]:
# Your Own Implementation Space
# Use this cell to implement your own unique agent system

# TODO: Replace this with your own creative implementation!

# Example starting points:
# 1. Travel Planning Agency
# travel_budget_agent = CustomSpecializedAgent("BudgetMaster", "travel budgeting", "practical and money-saving")
# travel_activity_agent = CustomSpecializedAgent("ActivityFinder", "local activities and attractions", "adventurous and cultural")

# 2. Health & Wellness Team  
# nutrition_agent = CustomSpecializedAgent("NutritionCoach", "nutrition and meal planning", "evidence-based and supportive")
# fitness_agent = CustomSpecializedAgent("FitnessTrainer", "exercise and physical wellness", "motivational and adaptive")

# 3. Creative Writing Studio
# plot_agent = CustomSpecializedAgent("PlotArchitect", "story structure and plot development", "creative and analytical")
# character_agent = CustomSpecializedAgent("CharacterCreator", "character development and dialogue", "psychological and empathetic")

# Your implementation here:
print("This is your space to implement your own agent system!")
print("Uncomment and modify the examples above, or create something entirely new!")

# Create your agents
# your_agent_1 = ...
# your_agent_2 = ...
# your_agent_3 = ...

# Create your orchestrator
# your_orchestrator = GroupChatOrchestrator([your_agent_1, your_agent_2, your_agent_3])

# Test your system
# await your_orchestrator.group_chat("Your test question here")

### Production Considerations

In [ ]:
# Production-Ready Considerations
# Important aspects to consider when deploying agents to production

class ProductionAgent(MemoryEnabledAgent):
    def __init__(self, name: str, description: str, max_retries: int = 3, timeout: int = 30):
        super().__init__(name, description)
        self.max_retries = max_retries
        self.timeout = timeout
        self.error_log = []
        
    async def safe_chat(self, message: str) -> Dict[str, Any]:
        """Production-safe chat with error handling and logging"""
        
        for attempt in range(self.max_retries):
            try:
                # Implement timeout
                response = await asyncio.wait_for(
                    self.chat_with_memory(message), 
                    timeout=self.timeout
                )
                
                return {
                    "success": True,
                    "response": response,
                    "attempt": attempt + 1,
                    "timestamp": datetime.now().isoformat()
                }
                
            except asyncio.TimeoutError:
                error_msg = f"Timeout after {self.timeout}s on attempt {attempt + 1}"
                self.error_log.append(error_msg)
                if attempt == self.max_retries - 1:
                    return {
                        "success": False,
                        "error": "Request timeout",
                        "attempts": self.max_retries,
                        "timestamp": datetime.now().isoformat()
                    }
                    
            except Exception as e:
                error_msg = f"Error on attempt {attempt + 1}: {str(e)}"
                self.error_log.append(error_msg)
                if attempt == self.max_retries - 1:
                    return {
                        "success": False,
                        "error": str(e),
                        "attempts": self.max_retries,
                        "timestamp": datetime.now().isoformat()
                    }
        
        return {
            "success": False,
            "error": "Max retries exceeded",
            "attempts": self.max_retries,
            "timestamp": datetime.now().isoformat()
        }

    def get_health_status(self) -> Dict[str, Any]:
        """Get agent health and performance metrics"""
        return {
            "agent_name": self.name,
            "total_conversations": len(self.conversation_history),
            "recent_errors": len([e for e in self.error_log if 'recent' in str(e)]),
            "last_activity": self.conversation_history[-1]["timestamp"] if self.conversation_history else None,
            "status": "healthy" if len(self.error_log) < 5 else "degraded"
        }

# Production considerations checklist
production_checklist = {
    "Security": [
        "API key rotation and secure storage",
        "Input validation and sanitization", 
        "Rate limiting and abuse prevention",
        "Audit logging for compliance"
    ],
    "Reliability": [
        "Error handling and graceful degradation",
        "Retry logic with exponential backoff",
        "Circuit breaker patterns",
        "Health checks and monitoring"
    ],
    "Performance": [
        "Response time optimization",
        "Caching strategies",
        "Connection pooling", 
        "Load balancing"
    ],
    "Scalability": [
        "Horizontal scaling design",
        "Database optimization",
        "Memory management",
        "Asynchronous processing"
    ],
    "Monitoring": [
        "Application performance monitoring",
        "Error tracking and alerting",
        "Usage analytics",
        "Cost monitoring"
    ]
}

print("Production Agent Example Created!")
print("\nProduction Readiness Checklist:")
print("=" * 40)

for category, items in production_checklist.items():
    print(f"\n{category}:")
    for item in items:
        print(f"  - {item}")

print(f"\nKey Production Features Implemented:")
print("- Error handling with retries")
print("- Timeout protection") 
print("- Health status monitoring")
print("- Detailed logging")

# Test production features
production_agent = ProductionAgent(
    name="ProdAgent",
    description="A production-ready agent with enhanced error handling",
    max_retries=2,
    timeout=10
)

print(f"\nProduction agent created: {production_agent.name}")
print(f"Max retries: {production_agent.max_retries}")
print(f"Timeout: {production_agent.timeout}s")

In [ ]:
# Test Production Features
# Demonstrate production-ready error handling and monitoring

async def test_production_features():
    """Test production agent capabilities"""
    
    print("Testing Production Agent Features")
    print("=" * 40)
    
    # Test successful interaction
    print("Test 1: Normal operation")
    result1 = await production_agent.safe_chat("What is machine learning?")
    print(f"Success: {result1['success']}")
    if result1['success']:
        print(f"Response: {result1['response'][:100]}...")
    print()
    
    # Test health status
    print("Test 2: Health status check")
    health = production_agent.get_health_status()
    print(f"Agent Status: {health['status']}")
    print(f"Total Conversations: {health['total_conversations']}")
    print(f"Recent Errors: {health['recent_errors']}")
    print()
    
    # Test with a complex question
    print("Test 3: Complex interaction")
    complex_question = "Explain the differences between supervised and unsupervised learning with examples"
    result2 = await production_agent.safe_chat(complex_question)
    print(f"Success: {result2['success']}")
    print(f"Attempts needed: {result2.get('attempt', 'N/A')}")
    print()
    
    # Show final health status
    print("Final Health Check:")
    final_health = production_agent.get_health_status()
    for key, value in final_health.items():
        print(f"  {key}: {value}")

# Run production tests
await test_production_features()

In [ ]:
# Workshop Summary and Next Steps
# Congratulations! You've completed the Semantic Kernel Agents Workshop

print("Semantic Kernel Agents Workshop - COMPLETED!")
print("=" * 50)

# Workshop achievements
achievements = [
    "✓ Set up Azure OpenAI with Semantic Kernel",
    "✓ Created basic conversational agents", 
    "✓ Built specialized agents with distinct roles",
    "✓ Implemented multi-agent group chat coordination",
    "✓ Added memory capabilities to agents",
    "✓ Explored production considerations",
    "✓ Built your own custom agent system"
]

print("\nWhat You've Accomplished:")
for achievement in achievements:
    print(f"  {achievement}")

# Key concepts learned
concepts = [
    "Semantic Kernel framework basics",
    "Azure OpenAI service integration", 
    "Agent specialization patterns",
    "Group chat orchestration",
    "Memory and context management",
    "Production readiness patterns"
]

print(f"\nKey Concepts Mastered:")
for concept in concepts:
    print(f"  • {concept}")

# Next steps for continued learning
next_steps = [
    "Experiment with different agent personalities and roles",
    "Integrate external APIs and data sources",
    "Build domain-specific agent ecosystems", 
    "Implement advanced memory and learning patterns",
    "Deploy agents to production environments",
    "Explore Semantic Kernel plugins and extensions",
    "Join the Semantic Kernel community for advanced techniques"
]

print(f"\nNext Steps for Advanced Development:")
for i, step in enumerate(next_steps, 1):
    print(f"  {i}. {step}")

# Resources for continued learning
resources = {
    "Semantic Kernel Documentation": "https://learn.microsoft.com/semantic-kernel/",
    "Azure OpenAI Service": "https://azure.microsoft.com/products/ai-services/openai-service",
    "Semantic Kernel GitHub": "https://github.com/microsoft/semantic-kernel",
    "Azure AI Foundry": "https://azure.microsoft.com/products/ai-foundry/"
}

print(f"\nAdditional Resources:")
for name, url in resources.items():
    print(f"  • {name}: {url}")

print(f"\n" + "=" * 50)
print("Thank you for completing the Semantic Kernel Agents Workshop!")
print("You're now ready to build sophisticated AI agent systems!")
print("=" * 50)

In [ ]:
# Final Demo: Complete Agent System
# Run this cell to see all components working together

async def final_demo():
    """Demonstrate the complete agent system capabilities"""
    
    print("FINAL DEMO: Complete Semantic Kernel Agent System")
    print("=" * 60)
    
    # Show available agents
    print("Available Agents:")
    all_agents = [
        ("Generic Agent", "General purpose conversational AI"),
        ("Tech Expert", "Software engineering and technical guidance"),
        ("Business Expert", "Strategy and business analysis"),
        ("Creative Expert", "Creative writing and marketing"),
        ("Memory Expert", "AI with persistent conversation memory"),
        ("Production Agent", "Enterprise-ready with error handling")
    ]
    
    for i, (name, desc) in enumerate(all_agents, 1):
        print(f"  {i}. {name}: {desc}")
    
    print(f"\nDemo Scenario: Multi-perspective analysis")
    demo_question = """
    Our company wants to implement AI chatbots for customer service. 
    What should we consider from technical, business, and creative perspectives?
    """
    
    print(f"Question: {demo_question}")
    print("-" * 60)
    
    try:
        # Run group chat with multiple agents
        result = await orchestrator.group_chat(demo_question, max_agents=3)
        
        print(f"\n" + "=" * 60)
        print("DEMO COMPLETED SUCCESSFULLY!")
        print("=" * 60)
        print(f"Agents Consulted: {', '.join(result['selected_agents'])}")
        print(f"Timestamp: {result['timestamp']}")
        
        print(f"\nThis demonstrates:")
        print("• Multi-agent collaboration")
        print("• Intelligent agent selection") 
        print("• Comprehensive response synthesis")
        print("• Production-ready architecture")
        
    except Exception as e:
        print(f"Demo error: {e}")

# Run the final demonstration
await final_demo()

In [ ]:
# Bonus: Quick Agent Creator
# Use this utility to quickly create and test new agents

def create_quick_agent(name: str, role: str, expertise: str, personality: str = "helpful"):
    """Quickly create a new specialized agent"""
    agent = SpecializedAgent(name, role, expertise, personality)
    print(f"Created {name} - {role} specializing in {expertise}")
    return agent

async def quick_test(agent, question: str):
    """Quickly test an agent with a question"""
    print(f"\nTesting {agent.name}:")
    print(f"Q: {question}")
    response = await agent.chat(question)
    print(f"A: {response}")
    return response

# Example: Create and test a new agent quickly
print("Quick Agent Creator - Create any specialist you need!")
print("=" * 50)

# Create a new specialist (modify these as needed)
data_scientist = create_quick_agent(
    name="DataWizard",
    role="Senior Data Scientist", 
    expertise="data analysis, statistics, and machine learning model development",
    personality="analytical and detail-oriented"
)

# Quick test
test_question = "How do I evaluate the performance of a classification model?"
await quick_test(data_scientist, test_question)

print(f"\nYour turn! Modify the parameters above to create any type of specialist agent.")
print("Try creating agents for: Legal advice, Medical information, Financial planning, etc.")
print("(Remember to stay within ethical and legal boundaries for sensitive domains)")

print(f"\n" + "=" * 50)
print("END OF WORKSHOP - Happy Agent Building!")
print("=" * 50)